In [5]:
using ForwardDiff, PyPlot

phi(r) = r^(-12) - 2 * r^(-6)
dphi(r) = -12 * r^(-13) + 12 * r^(-7)
ddphi(r) = 12*13 * r^(-14) - 7*12 * r^(-8)
pphi(r) = (12*13 - 7*12) * r^(-8)

function energy(X::Matrix)
    E = 0.0 
    nX = size(X,2)
    for i = 1:nX-1, j = i+1:nX
        rij = norm(X[:,i] - X[:,j])
        E += phi(rij)
    end 
    return E
end 

dofs2pos(x::Vector) = reshape(x, 2, length(x) ÷ 2) 
pos2dofs(X::Matrix) = X[:]

energy(x::Vector) = energy(dofs2pos(x))

function gradient(X::Matrix)
    dE = zeros(size(X))
    nX = size(X,2)
    for i = 1:nX-1, j = i+1:nX 
        Rij = X[:,i] - X[:,j]
        rij = norm(Rij)
        a = dphi(rij) * Rij / rij 
        dE[:,i] += a 
        dE[:,j] -= a 
    end 
    return dE
end 

gradient(x::Vector) = pos2dofs(gradient(dofs2pos(x)))

function exp_precond(x; rcut = 2.5, α=3.0)
   X = dofs2pos(x)
   I = dofs2pos(1:length(x) |> collect)
   nX = size(X, 2)
   P = zeros(2*nX, 2*nX)
   for i = 1:nX, j = i+1:nX
      Ii = I[:,i]; Ij = I[:,j]
      Rij = X[:,i] - X[:,j]
      rij = norm(Rij)
      if 0 < rij < rcut
         a = pphi(rij) * eye(2)
         P[Ii, Ij] -= a
         P[Ij, Ii] -= a
         P[Ii, Ii] += a
         P[Ij, Ij] += a
      end
   end
   return sparse(P) + 0.001 * speye(2*nX)
end

function refconfig(; R = 4.1, case=:min, rattle=0.01)
   A = [1.0 cos(π/3); 0.0 sin(π/3)]
   cR = ceil(Int, R / minimum(svd(A)[2]))
   t = collect(-cR:cR)
   x = ones(length(t)) * t'
   y = t * ones(length(t))'
   X = A * [x[:] y[:]]'
   r = sqrt(sumabs2(X, 1))
   Xref = X[:, find(0 .< r .<= R)]
   r = sqrt(sumabs2(Xref, 1))
   I0 = find(r .<= 1.1)[1]
   if I0 != 1
      Xref[:, [1,I0]] = Xref[:, [I0,1]]
      r[[1,I0]] = r[[I0,1]]
   end
    
   if case == :min 
        Xref[:, 1] *= 0.9 
    elseif case == :saddle
        Xref[:, 1] *= 0.6 
    end 
    Xref += 2 * rattle * (rand(size(Xref)) - 0.5)
   return Xref
end

function test_gradient()
    x = refconfig(3.1) |> pos2dofs
    N = length(x)
    x += 0.01 * rand(N)
    E = energy(x)
    dE = gradient(x)
    dEh = [] 
    for p = 3:10
        h = 0.1^p
        dEh = zeros(N)
        for i = 1:N
            x[i] += h
            dEh[i] = (energy(x) - E) / h
            x[i] -= h
        end
        @show norm(dE - dEh, Inf)
    end
end


function sd(; R = 3.1, precond = x -> speye(length(x)), α = 0.001, TOL = 1e-5, maxnit = 1000, verbose=false)
    x = refconfig(R=R, rattle=0.0) |> pos2dofs 
    for n = 1:maxnit 
        g = gradient(x)
        if verbose
            @printf(" %d  |  %4.2e  \n", n, norm(g, Inf))
        end 
        if norm(g, Inf) <= TOL
            println("success after $n iterations")
            return x
        end
        x = x - α * (precond(x) \ g)
    end 
    println("sd did not converge in $maxnit iterations")
    return x
end


function bb(; R = 3.1, precond = x -> speye(length(x)), α = 0.001, TOL = 1e-5, maxnit = 1000, verbose=false)

    x = refconfig(R=R, rattle=0.0) |> pos2dofs    
    
    g = gradient(x) 
    gP = precond(x) \ g
    xold = x
    x = xold - α * gP
    
    for n = 1:maxnit 
        gold = g
        gPold = gP
        g = gradient(x)
        
        if verbose
            @printf(" %d  |  %4.2e  \n", n, norm(g, Inf))
        end 
        if norm(g, Inf) <= TOL
            println("success after $n iterations")
            return x
        end
        
        gP = precond(x) \ g
        α = dot(x - xold, g - gold) / dot(g - gold, gP - gPold)

        xold = x 
        x = xold - α * gP
    end 
    println("sd did not converge in $maxnit iterations")
    return x
end


function dimer(; R = 3.1, precond = x -> speye(length(x)), 
                 α = 0.001, β = 0.001, TOL = 1e-5, TOLr = 1e-3, 
                 maxnit = 2000, verbose=false)
    x = refconfig(R=R, case=:saddle, rattle=0.0) |> pos2dofs 
    v = zeros(x)
    v[1:2] = - x[1:2]
    l = 1e-3
    
    for n = 1:maxnit
        P = precond(x)
        v /= sqrt(dot(v, P*v))
        gp = gradient(x + l * v)
        gm = gradient(x - l * v)
        gavg = 0.5 * (gp + gm)
        Hxv = (gp - gm) / (2*l)

        res_t = norm(gavg, Inf)
        res_r = norm(Hxv - dot(v, Hxv) * (P * v), Inf)
        
        if verbose
            @printf(" %d  |  %4.2e   %4.2e \n", n, res_t, res_r)
        end 
        
        if res_t <= TOL && res_r <= TOLr
            println("success after $n iterations")
            return x, v
        end
        
        x = x - α * (P \ gavg - 2 * dot(v, gavg) * v)
        v = v - β * (P \ Hxv - dot(v, Hxv) * v)
    end 
    println("dimer did not converge in $maxnit iterations")
    return x, v
end


function bbdimer(; R = 3.1, precond = x -> speye(length(x)), 
                 α = 0.001, β = 0.001, TOL = 1e-5, TOLr = 1e-3, 
                 maxnit = 2000, verbose=false)

    x = refconfig(R=R, case=:saddle, rattle=0.0) |> pos2dofs 
    v = zeros(x)
    v[1:2] = - x[1:2]
    l = 1e-3
        
    P = precond(x)
    v /= sqrt(dot(v, P*v))
    gp = gradient(x + l * v)
    gm = gradient(x - l * v)
    gavg = 0.5 * (gp + gm)
    Hxv = (gp - gm) / (2*l)
    Ft = gavg - 2 * dot(v, gavg) * (P * v)
    Fr =  Hxv - dot(v, Hxv) * (P * v)
    FtP = P \ gavg - 2 * dot(v, gavg) * v
    FrP = P \ Hxv - dot(v, Hxv) * v

    x_old, v_old = x, v
    x = x_old - α * FtP
    v = v_old - β * FrP

    for nit = 1:maxnit 

        # store previous state
        Ft_old, Fr_old, FtP_old, FrP_old = Ft, Fr, FtP, FrP 
        
        # update quantities 
        P = precond(x)
        v /= sqrt(dot(v, P*v))
        gp = gradient(x + l * v)
        gm = gradient(x - l * v)
        gavg = 0.5 * (gp + gm)
        Hxv = (gp - gm) / (2*l)
        λ = dot(Hxv, v)
        Ft = gavg - 2 * dot(v, gavg) * (P * v)
        Fr =  Hxv - λ * (P * v)
        
        rest = norm(Ft, Inf) 
        resr = norm(Fr, Inf)
        
        if verbose
            @printf("%4d | %1.2e  %1.2e  %1.2e  %1.2e  %1.2e \n",
               nit, rest, resr, λ, α, β)        
        end 
        if rest <= TOL && resr <= TOLr 
            println("success after $nit iterations")
            return x, v
        end
        
        FtP = P \ gavg - 2 * dot(v, gavg) * v
        FrP = P \ Hxv - dot(v, Hxv) * v
        
        α = dot(x - x_old, Ft - Ft_old) / dot(FtP - FtP_old, Ft - Ft_old)
        β = dot(v - v_old, Fr - Fr_old) / dot(FrP - FrP_old, Fr - Fr_old)
        
        x_old, v_old = x, v
        x = x_old - α * FtP 
        v = v_old - β * FrP
    end
    println("dimer did not converge in $maxnit iterations") 
    return x, v
end


bbdimer (generic function with 1 method)

In [3]:
println("MINIMISATION")
for R in (3.1, 4.1, 5.1, 7.1) # , 10.1)
    println("R = $R")
    print("SD-I: ")
    xI = sd(R = R, α=0.003, verbose = false);
    print("SD-P: ")
    xP = sd(R = R, precond = exp_precond, α = 1.0, verbose = false);
    print("BB-I: ")
    xIbb = bb(R = R, α=0.001, verbose = false);
    print("BB-P: ")
    xPbb = bb(R = R, α=0.1, verbose = false, precond = exp_precond);
end 

MINIMISATION
R = 3.1
SD-I: success after 309 iterations
SD-P: success after 98 iterations
BB-I: success after 55 iterations
BB-P: success after 24 iterations
R = 4.1
SD-I: success after 364 iterations
SD-P: success after 89 iterations
BB-I: success after 62 iterations
BB-P: success after 24 iterations
R = 5.1
SD-I: success after 432 iterations
SD-P: success after 85 iterations
BB-I: success after 81 iterations
BB-P: success after 24 iterations
R = 7.1
SD-I: success after 615 iterations
SD-P: success after 83 iterations
BB-I: success after 87 iterations
BB-P: success after 23 iterations


In [6]:
println("SADDLE SEARCH")
for R in (3.1, 4.1, 5.1, 7.1)  # , 10.1)
    println("R = $R")
    print("  DIMER-I: ")
    xI = dimer(; R=R, verbose=false, α=0.0015, β = 0.001);
    print("  DIMER-P: ")
    xP = dimer(; R=R, verbose=false, α=0.9, β = 0.3, precond=exp_precond);
    print("BBDIMER-I: ")
    xIbb = bbdimer(R = R, α=0.001, β=0.001, verbose = false);
    print("BBDIMER-P: ")
    xPbb = bbdimer(R = R, α=0.1, β=0.1, verbose = false, precond = exp_precond);
end 

SADDLE SEARCH
R = 3.1
  DIMER-I: success after 650 iterations
  DIMER-P: success after 114 iterations
BBDIMER-I: success after 76 iterations
BBDIMER-P: success after 31 iterations
R = 4.1
  DIMER-I: success after 852 iterations
  DIMER-P: success after 107 iterations
BBDIMER-I: success after 69 iterations
BBDIMER-P: success after 29 iterations
R = 5.1
  DIMER-I: success after 1133 iterations
  DIMER-P: success after 106 iterations
BBDIMER-I: success after 96 iterations
BBDIMER-P: success after 27 iterations
R = 7.1
  DIMER-I: success after 1958 iterations
  DIMER-P: success after 105 iterations
BBDIMER-I: success after 175 iterations
BBDIMER-P: success after 29 iterations
